In [ ]:
from localtileserver import get_leaflet_tile_layer
from ipyleaflet import (Map, LayersControl, LayerGroup, basemaps, 
                        basemap_to_tiles, MeasureControl)
from ipywidgets import Layout, Dropdown

# base maps
mapnik = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
mapnik.base = True
mapnik.name = 'Mapnik Layer'

esri = basemap_to_tiles(basemaps.Esri.WorldImagery)
esri.base = True
esri.name = 'ESRI Layer'

bzh = basemap_to_tiles(basemaps.OpenStreetMap.BZH)
bzh.base = True
bzh.name = 'BZH layer'

# First, create a tile server from local raster file
tile_client = TileClient(r"C:\BERATools\Test-data\CHM.tif")
chm = get_leaflet_tile_layer(tile_client)
chm.name = "CHM"

# Study sites
tile_4203 = TileClient(r"C:\BERATools\Test-data\rasters\P1_4203_transparent_mosaic_Shifted.tif")
t_4203 = get_leaflet_tile_layer(tile_4203)
t_4203.name = "Site 4203"

tile_4207 = TileClient(r"C:\BERATools\Test-data\rasters\P1_4207_transparent_mosaic_Shifted.tif")
t_4207 = get_leaflet_tile_layer(tile_4207)
t_4207.name = "Site 4207"

tile_4209 = TileClient(r"C:\BERATools\Test-data\rasters\P1_4209_transparent_mosaic_Shifted.tif")
t_4209 = get_leaflet_tile_layer(tile_4209)
t_4209.name = "Site 4209"

chm.base = False
t_4203.base = False
t_4207.base = False
t_4209.base = False

m = Map(layers=[mapnik, esri, bzh],
        center=tile_client.center(),
        zoom=tile_client.default_zoom,
        scroll_wheel_zoom=True,
        layout=Layout(height='750px'))

m.add(chm)
m.add(t_4203)
m.add(t_4207)
m.add(t_4209)

# measure control
measure = MeasureControl(
    position='topright',
    active_color = 'orange',
    primary_length_unit = 'meters'
)
m.add(measure)

measure.completed_color = 'red'

measure.add_length_unit('yards', 1.09361, 4)
measure.secondary_length_unit = 'yards'

measure.add_area_unit('sqyards', 1.19599, 4)
measure.secondary_area_unit = 'sqyards'

m.add(LayersControl())


In [ ]:
import json
import folium
import requests

from localtileserver import get_folium_tile_layer
from localtileserver import TileClient

tile_client = TileClient(r"C:\BERATools\Test-data\CHM.tif")
chm = get_folium_tile_layer(tile_client, max_zoom=24)
chm.layer_name = "CHM"
chm.overlay = True

url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
antarctic_ice_edge = f"{url}/antarctic_ice_edge.json"
antarctic_ice_shelf_topo = f"{url}/antarctic_ice_shelf_topo.json"

folium.folium._default_css.append(('leaflet_overloaded_css', 'https://your_url/your_css_file.css'))

m = folium.Map(
    location=[-59.1759, -11.6016],
    tiles=None,
    zoom_start=2,
)

tile_layer = folium.TileLayer(
    tiles="https://{s}.basemaps.cartocdn.com/rastertiles/dark_all/{z}/{x}/{y}.png",
    attr='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>',
    max_zoom=19,
    name='darkmatter',
    control=True,
    opacity=0.7
)
tile_layer.add_to(m)

chm.add_to(m, name='chm')

folium.GeoJson(antarctic_ice_edge, name="geojson").add_to(m)

folium.TopoJson( 
    json.loads(requests.get(antarctic_ice_shelf_topo).text), 
    "objects.antarctic_ice_shelf", 
    name="topojson", 
).add_to(m) 

folium.LayerControl().add_to(m)

m